# Proyecto 2: Primera parte
Curso Ingeniería de Características


**Estephania Pivac Alcaraz**

---

**Problema de interés:**
Estudar la relación que existe entre la cantidad de accidentes de tránsito y la inversión a la conservación de carreteras y presupuesto gubernamental.

## 1. Abriendo las bases de datos

Abrimos las bases de datos que estaremos utilizando. Para esto construiremos los dataframes necesarios así como también limpiaremos y analizaremos dichos datos.

Comenzamos importando librerías que necesitaremos

In [1]:
# Instalamos Pandas-profiling
#pip install Pandas-Profiling

# Importamos algunas librerías que vamos a necesitar
import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
# import requests # Otra forma no de las librerías de uso comun

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import zipfile # Descompresión de archivos
import numpy as np
from pandas_profiling import ProfileReport

### 1.1 Tidy_accidentes: Base de datos de accidentes de tránsito

Abrimos el archivo con la información de los accidentes de tránsito en México desde 2017 hasta 2021. Generamos el dataframe con el que estaremos trabajando y convertimos cada columna al tipo de dato adecuado.

In [2]:
# Generamos el dataframe de accidentes
tidy_accidentes = pd.read_csv("tidy_accidentes.csv", index_col=False)

# Convertimos cada variable de acuerdo a su tipo
# Categoricas
tidy_accidentes = tidy_accidentes.astype({'TIPACCID':'category', 'CAUSAACCI':'category', 'CAPAROD':'category', 'CLASACC':'category', 'EDAD':'category'})

# Fechas
tidy_accidentes['FECHA'] = pd.to_datetime(tidy_accidentes['FECHA'], format='%Y/%m/%d', errors='coerce')
tidy_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81848 entries, 0 to 81847
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID_ENTIDAD  81848 non-null  int64         
 1   ANIO        81848 non-null  int64         
 2   TIPACCID    81848 non-null  category      
 3   CAUSAACCI   81848 non-null  category      
 4   CAPAROD     81848 non-null  category      
 5   CLASACC     81848 non-null  category      
 6   FECHA       81848 non-null  datetime64[ns]
 7   EDAD        81848 non-null  category      
dtypes: category(5), datetime64[ns](1), int64(2)
memory usage: 2.3 MB


Observemos la estructura de la base de datos:

In [3]:
tidy_accidentes.head()

,ID_ENTIDAD,ANIO,TIPACCID,CAUSAACCI,CAPAROD,CLASACC,FECHA,EDAD
0,1,2021,Salida del camino,Conductor,Pavimentada,Sólo daños,2021-01-03,28-37
1,1,2021,Salida del camino,Conductor,Pavimentada,No fatal,2021-01-04,Menor de edad
2,1,2021,Colisión con vehículo automotor,Conductor,Pavimentada,Sólo daños,2021-01-17,48-57
3,1,2021,Salida del camino,Conductor,Pavimentada,Sólo daños,2021-01-26,Se fugó
4,1,2021,Colisión con vehículo automotor,Conductor,Pavimentada,Sólo daños,2021-01-28,38-47


Veamos que esta base de datos no contiene datos faltantes:

In [4]:
tidy_accidentes.isnull().sum()

ID_ENTIDAD    0
ANIO          0
TIPACCID      0
CAUSAACCI     0
CAPAROD       0
CLASACC       0
FECHA         0
EDAD          0
dtype: int64

### 1.2 Tidy conservación: Base de datos de conservación de carreteras del año 2019

Abrimos el archivo csv y lo guardamos en formato de dataframe

In [5]:
# Generamos el dataframe de conservación
tidy_conservacion = pd.read_csv("tidy_conservacion.csv", index_col=False)
tidy_conservacion

# Convertimos cada variable de acuerdo a su tipo
# Categoricas
tidy_conservacion = tidy_conservacion.astype({'ENTIDAD':'category', 'PROGRAMA_CONSERVACION':'category'})
tidy_conservacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1859 entries, 0 to 1858
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   ID_ENTIDAD             1859 non-null   int64   
 1   ENTIDAD                1859 non-null   category
 2   TOTAL_CONSERVACION     1859 non-null   float64 
 3   PROGRAMA_CONSERVACION  1859 non-null   category
dtypes: category(2), float64(1), int64(1)
memory usage: 34.7 KB


In [6]:
tidy_conservacion.isnull().sum()

ID_ENTIDAD               0
ENTIDAD                  0
TOTAL_CONSERVACION       0
PROGRAMA_CONSERVACION    0
dtype: int64

In [7]:
tidy_conservacion

,ID_ENTIDAD,ENTIDAD,TOTAL_CONSERVACION,PROGRAMA_CONSERVACION
0,9,CIUDAD DE MEXICO,1.294363e+08,Estudios y/o Proyectos
1,9,CIUDAD DE MEXICO,1.091483e+08,Programa Asociación Público Privada de Conserv...
2,9,CIUDAD DE MEXICO,4.997081e+07,Programa Asociación Público Privada de Conserv...
3,9,CIUDAD DE MEXICO,8.514464e+07,Programa Asociación Público Privada de Conserv...
4,9,CIUDAD DE MEXICO,2.415727e+07,Programa Asociación Público Privada de Conserv...
...,...,...,...,...
1854,32,ZACATECAS,6.094561e+06,Conservación Rutinaria
1855,32,ZACATECAS,5.716601e+06,Conservación Rutinaria
1856,32,ZACATECAS,1.633732e+06,Otros Subprogramas
1857,32,ZACATECAS,2.625802e+06,Otros Subprogramas


In [8]:
tidy_conservacion['PROGRAMA_CONSERVACION'].value_counts()

Conservación Periódica                                  721
Conservación Rutinaria                                  632
Señalamiento                                            234
Reconstrucción                                          125
Otros Subprogramas                                       80
Coparticipación FONDEN                                   26
Conservación Rutinaria                                   14
Programa Asociación Público Privada de Conservación      11
Programa Asociación Público Privada de Conservación       7
Señalamiento                                              4
Contratos Plurianuales de Conservación de Carreteras      2
Reconstrucción                                            2
Estudios y/o Proyectos                                    1
Name: PROGRAMA_CONSERVACION, dtype: int64

### 1.3 Budget transparency: Base de datos sobre el presupuesto público aprobado de 2013 a 2021

In [9]:
# Generamos el dataframe de presupuesto
df_budget = pd.read_csv("budget_transparency.csv", index_col=False)
df_budget

# Convertimos cada variable de acuerdo a su tipo
# Categoricas
df_budget = df_budget.astype({'State':'category', 'Expenses Type ID':'category'})

# Convertimos Year a datetime
df_budget['Year'] = pd.to_datetime(df_budget['Year'], format='%Y', errors='coerce')

df_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   State ID          288 non-null    int64         
 1   State             288 non-null    category      
 2   Year              288 non-null    datetime64[ns]
 3   Expenses Type ID  288 non-null    category      
 4   Expenses Type     288 non-null    object        
 5   Amount Approved   288 non-null    int64         
dtypes: category(2), datetime64[ns](1), int64(2), object(1)
memory usage: 11.1+ KB


In [10]:
# Exploramos el dataframe
df_budget

,State ID,State,Year,Expenses Type ID,Expenses Type,Amount Approved
0,1,Aguascalientes,2013-01-01,8,Gasto de Obra Pública,1511908040
1,1,Aguascalientes,2014-01-01,8,Gasto de Obra Pública,1865135779
2,1,Aguascalientes,2015-01-01,8,Gasto de Obra Pública,1659723400
3,1,Aguascalientes,2016-01-01,8,Gasto de Obra Pública,1717532284
4,1,Aguascalientes,2017-01-01,8,Gasto de Obra Pública,1230564370
...,...,...,...,...,...,...
283,32,Zacatecas,2017-01-01,8,Gasto de Obra Pública,1992505733
284,32,Zacatecas,2018-01-01,8,Gasto de Obra Pública,2284882371
285,32,Zacatecas,2019-01-01,8,Gasto de Obra Pública,2567673138
286,32,Zacatecas,2020-01-01,8,Gasto de Obra Pública,2010497486


In [11]:
# Observamos si existen datos faltantes
df_budget.isnull().sum()

State ID            0
State               0
Year                0
Expenses Type ID    0
Expenses Type       0
Amount Approved     0
dtype: int64

## 2. EDA automático utilizando Pandas-profiling
Análisis exploratiorio de datos automático generado en html.

### 2.1 EDA: Tidy_accidentes

In [ ]:
# Generamos el reporte en formato html usando pandas-profiling
prof = ProfileReport(tidy_accidentes)
prof.to_file(output_file='EDA_accientes.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

### 2.2 EDA: Tidy_conservacion

In [ ]:
# Generamos el reporte en formato html usando pandas-profiling
prof = ProfileReport(tidy_conservacion)
prof.to_file(output_file='EDA_conservacion.html')

### 2.3 EDA: df_budget

In [8]:
# Generamos el reporte en formato html usando pandas-profiling
prof = ProfileReport(df_budget)
prof.to_file(output_file='EDA_budget.html')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   State ID          288 non-null    int64   
 1   State             288 non-null    category
 2   Year              288 non-null    int64   
 3   Expenses Type ID  288 non-null    category
 4   Expenses Type     288 non-null    object  
 5   Amount Approved   288 non-null    int64   
dtypes: category(2), int64(3), object(1)
memory usage: 11.1+ KB


## 3. EDA orientado a relación entre variables

Abordaremos variables más específicas para estudiar la relación que nos interesa.

### 3.1 Accidentes: frecuencia, tipo y causa

In [ ]:
df_accidentes['CAUSAACCI'].value_counts()